# 16024652 - Final Task - 13/01/2017

## Simulating an Angry Birds Game


The aim of this task is to create a code which allows a user to launch a projectile at a target by inputting a speed and an angle. The user must try to knock the target over by varying this angle and initial speed. 

Classical mechanics will be used to calculate the porperties of both the projectile and the target dependant on the angle and speed the user inputs, and therefore their resulting interaction. This will be used to define functions for the torque and projectile calculations which in an animation will show the motion of the whole sequence of events. If and while loops will be used extensively to redraw objects on the canvas creating the animation. 

Here necessary libraries are being imported, occasionally with useful alias's that will save time and space in the code

In [1]:
import numpy as np
from vpython import sys, compound, sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, rotate, time, sleep, textures, ellipsoid,cone

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Part 1: Essential Values

These values are all values that are needed throughout the script. It is useful to define them with alias's if they are to be used multiple times because it makes it more clear what the term is, as well as saving space and time in the code. 

The projectile has now been named 'bird' in fitting with the theme of an Angry Birds game.

In [2]:
x0 = 0.0                     # initial x-value
y0 = 0.0                     # initial y-value
t = 0             
dt = 0.01                    # change in time
g = 9.81                     # acceleration due to gravity

birdm = 0.1                  # mass of the bird

Tm = 100                     # target mass
Th = 2                       # target height in y-direction
Tl = 0.5                     # target length in z-direction
Tw = 0.5                     # target width in x-direction
Tx = random()*10+5           # x-position of target, this code creates a random number between 5 and 15
dr = (Tw)/2                  # horizontal distance between the point of rotation and the centre of mass
hit_tolerance = 0.05+dr      # defined as sum of the radius of the bird (0.05m) and the distance from the edge 
                             # of the target to the central vertical axis of the target

### Part 2: The Torque Condition

Here we aim to find if the torque provided by the projectile is sufficient to overcome the restoring torque, which operates through the centre of mass. This would rotate the target clockwise (assuming the projectile is arriving from the left) around its bottom right corner, which is referred to as the point of rotation in the code. 

First we consider the torque which will try to resore the target to it's normal, stable, upright position:

$$\tau _{restoring}=\boldsymbol{F}_{grav} \times \mathbf{d}_{r}$$

where $\mathbf{F}_{grav} = m\boldsymbol{g}$ is the force provided by gravity, operating through the centre of mass,
and $\mathbf{d}_{r}=\frac{w}{2}$ is the horizontal distance between the point of rotation (i.e. the bottom
right corner) and the centre of mass.

We are interested in the magnitude of this torque so: 

$$\left | \tau _{restoring} \right |=-mg\frac{w}{2}$$

We must also consider the torque supplied by the collision:

$$\tau _{applied}=\boldsymbol{F}_{apploed} \times \mathbf{d}_{a}$$

where $\mathbf{d}_{a}$ is the vector from the point of rotation to the point of impact, on the left-hand side of the target at height h and $\mathbf{F}_{applied}=\frac{\boldsymbol{p}_{bird}}{\Delta t}$.

To find applied force we therefore need to find momentum of the bird which, as shown in the code, is worked out using:

$$p_{x}=mv_{0}cos\left ( \theta  \right )$$

$$p_{y}=mv_{0}sin\left ( \theta  \right )-mgt$$

The target will topple if:

$$\left | \tau _{applied} \right |> \left | \tau _{restoring} \right |$$

so this is the condition the following function is based around.

The variable hit tolerance is equal to the sum of the radius of the bird (5cm) and the distance from the edge of the target to the central vertical axis of the target. It is defined that if the distance between the bird’s position and the x-position of the target is less than hit tolerance, a collision has taken place.

In [3]:
def torque():
    '''function that calculates the torque condition.  
    Inputs:
    g: acceleration due to gravity
    dr: horizontal distance between the point of rotation and the centre of mass
    Tm: target mass
    v0: initial velocity inputted by user
    theta: initial angle inputted by user
    birdm: mass of bird
    t: time
    dt: timestep
    Returns:
    1: applied torque > restoring torque, the target topples
    0: applied torque < restoring torque, the target does not topple
    '''
    
    global Tor                                       # so Tor can be referenced outside this function
    Tor = abs(-g*Tm*dr)                              # finding the restoring torque               
    px = np.cos(theta)*birdm*v0                      # x-componant of momentum
    py = (np.sin(theta)*birdm*v0)- (birdm*g*t)       # y-componant of momentum
    global p                                         # so p can be referenced outside this function
    p = vector(px,py,0)                              # finding momentum vector
    rot = vector(T.pos.x+0.25,0,0)                   # point of rotation, bottom right hand corner of target 
    da = bird.pos - rot                        
    # vector from the point of rotation to the point of impact, on the left-hand side of the target at height h
    Fa = p/dt                                        # applied force
    global Toa                                       # so Toa can be referenced outside this function
    Toa = cross(Fa,da)                               # calculating applied torque
    
    
    if mag(Toa) > Tor:
        return 1
    else:
        return 0    

### Part 3: Defining Motion

Here the user is asked to input an initial speed and angle. From this the position of the bird with respect to time can be worked out using:

$$x = x_{0}+v_{0}tcos\left ( \theta  \right )$$
$$y = y_{0}+v_{0}tsin\left ( \theta  \right )-\frac{gt^{2}}{2}$$

This sends the bird into projectile motion. Whether the bird then hits the target or not depends on the values the user inputs. If the bird does not hit the target the user must try again. Simularly, if the bird does hit the target but the applied torque is not enough to overcome the restoring torque, the user must also try again. The user wins the game if the bird hits the target and the applied torque is enough to knock over the target.

In [16]:
def projectile():
    '''function that animates the projectile motion and collision.  
    Inputs:
    v0
    Returns:
    1: the bird hits the target and applied torque > restoring torque, the target topples
    0: the bird does not hit the target or it does but applied torque < restoring torque, the target does not topple
    '''
    global t             
    t=0
    global bird    # creating the compound bird object from an ellisoid, a sphere and a cone
    birdbody = ellipsoid(pos=vector(x0,y0,0), length=0.6, height=0.5, width=0.5,color=color.magenta)
    birdhead = sphere(pos=vector(0.4,0.25,0.25),radius = 0.2,color=color.magenta)
    birdbeak = cone(pos=vector(0.6,0.25,0.25),axis=vector(0.3,0,0),radius=0.1,color=color.orange)
    bird = compound([birdbody, birdhead,birdbeak], make_trail=True)
    
    #here the user is told to input terms 
    dtheta = float(input("Input the initial angle in degrees: "))
    global theta
    theta = float(np.radians(dtheta))
    global v0
    v0 = float(input("Input the initial speed in m/s: "))
    
    # while loop continues under the condition that the y position of the bird is greater than y0
    while y0+(v0*t*np.sin(theta))-(0.5*g*t**2) >= y0:
        t = t + dt                                        # increasing time by timestep
        rate(1/dt)                                        # defining the rate of the animation (real time)
        x = x0+v0*t*np.cos(theta)                         # calculating new x-position of bird
        y = y0+v0*t*np.sin(theta) - 0.5*g*t**2            # calculating new y-position of bird
        bird.pos = vector(x,y,0)                          # new vector position of bird
        if abs(bird.pos.x-T.pos.x) < hit_tolerance:       
            if 0 < y < 2.3:
                label(pos=vector(8,6,0),color=color.green,text="You have hit the target",box=False, background=color.black, opacity=1, height=20)
                if torque() == 1: 
                    label(pos=vector(8,5,0),color=color.green,text="The target has toppled", box=False, background=color.black, opacity=1, height =17)
                    label(pos=vector(8,-5,0),color=color.yellow,text="You win!",box=False,height=30, background=color.black, opacity=1, border=100)
                    #labelx= label(pos=vector(1,-6,0), text = '\n Impact height {0:0.2f} m \n Momentum {0:0.2f} kgm/s \n Applied torque {0:0.0f} Nm \n Restoring torque {0:0.0f} Nm'.format(bird.pos.y,mag(p),Toa,Tor))
                    print("The target has toppled")
                    print("Impact point height: {0:0.2f} m".format(bird.pos.y))
                    print("Momentum of bird at impact point: {0:0.2f} kgm/s".format(mag(p)))
                    print("Applied torque: {0:0.0f} Nm".format(mag(Toa)))
                    print("Magnitude of restoring torque: {0:0.0f} Nm".format(Tor))
                    sys.stdout.flush()
                    
                    i = 0
                    while i <= 10:
                        T.rotate(angle=0.1, origin=vector(T.pos.x+0.25,0,0), axis=vector(0,0,-1) ) 
                        i= i+0.01
                        if y >= 0.25:
                            x = bird.pos.x
                            y = bird.pos.y - 0.5*g*(i**2)
                            bird.pos = vector(x,y,0)
                            sleep(0.0001)
                    return 1
                elif torque() == 0:
                    label(pos=vector(8,-5,0),color=color.red,text="You lose, try again",box=False,height=30, background=color.black, opacity=1)
                    print("The target has not toppled")
                    print("Impact point height: {0:0.2f} m".format(bird.pos.y))
                    print("Momentum of bird at impact point: {0:0.2f} kgm/s".format(mag(p)))
                    print("Applied torque: {0:0.0f} Nm".format(mag(Toa)))
                    print("Magnitude of restoring torque: {0:0.0f} Nm".format(Tor))
                    sys.stdout.flush()
                    label(pos=vector(8,5,0),color=color.red,text="The target has not toppled",box=False, background=color.black, opacity=1)
                    bird.visible = False      # returning bird to start
                    bird.clear_trail()        # removing bird trail
                    return 0 
        elif abs(bird.pos.x-T.pos.x)>hit_tolerance and bird.pos.y < 0:
            print("The target has not been hit")
            label(pos=vector(8,-5,0),color=color.red,text="You lose, try again",box=False,height=30, background=color.black, opacity=1)
            label(pos=vector(8,6,0),color=color.red,text="You have not hit the target",box=False, background=color.black, opacity=1, height=17)
            sys.stdout.flush()
            bird.visible = False
            bird.clear_trail()
            return 0
    else:
        pass

### Part 4: Animating the Game

Now the canvas is set up such that there is a bird, target and ground on it. The functions can now be run such that the objects move in the ways calculated in the functions. In the projectile() function it was defined that if the target was toppled the function would return 1, and if it was either hit and not toppled or not hit at all it would return 0, therefore all we need to constrain here is that whilst x is still equal to 0 the animation (loops) carry on, and if not, they stop. 

In [17]:
scene = canvas(width=640, height=480, center=vector(8,0,0),range=8)
ground = curve(pos=[(0,0,0),(16,0,0)], color=color.green)
T = box(pos=vector(Tx,1,0), size=vector(Tl,Th,Tw), texture=textures.wood) #targer

x = 0
while x==0:
    x = projectile()
    


<IPython.core.display.Javascript object>

Input the initial angle in degrees: 10
Input the initial speed in m/s: 40
The target has toppled
Impact point height: 1.86 m
Momentum of bird at impact point: 3.95 kgm/s
Applied torque: 752 Nm
Magnitude of restoring torque: 245 Nm


### Part 5: Game Discussion

To imporove the representation of the physics in the game we mostly need to reduce the number of assumptions we make in our calculations.

Perhaps the biggest assumption that is made is that the collision between the ball and the target is completely elastic so no energy is lost. This means that the ball comes completely to rest transferring all its momentum to the target. In reality this is unlikely to happen and instead the ball would be more likely to bounce off the target slightly and bounce on the ground as it gains a small negative momentum from the collision.

The contact time between the ball and the target is finite, and is the same for all collisions. It can thus be represented as a fixed parameter ∆tcollision.


The target cannot slide on the ground;

The target has constant density, so that its centre of mass is at the geometric centre
of the target.

### References

I found out about the function sys.stdout.flush() from http://stackoverflow.com/questions/10019456/usage-of-sys-stdout-flush-method.

All other research from http://www.glowscript.org/docs/VPythonDocs/index.html.